In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
ls data/sports_scores/

astros_1A10.csv*   Rice_10H80.csv*    Tex_15E40.csv*
dynamo_10H30.csv*  rockets_1A10.csv*  UH_10H50.csv*


In [3]:
ls data/crime_beat/

crime_10H30_10_17.csv*  crime_10H80_10_17.csv*  crime_1A10_10_17.csv*
crime_10H50_10_17.csv*  crime_15E40_10_17.csv*


In [4]:
games = 'data/sports_scores/Rice_10H80.csv'
crime = 'data/crime_beat/crime_10H80_10_17.csv'

# scores df

In [5]:
df = pd.read_csv(games,index_col='Date').sort_index(ascending=True)

## win column
df['win'] =  np.where(df['Pts'] > df['Opp'],1,0)

# only oposing name and win boolean
# df = df[['Opponent','win','beat']]
df = df[['win','beat']]

# show

df.head()

,win,beat
Date,,
2010-09-04,0,10H80
2010-09-18,0,10H80
2010-09-25,0,10H80
2010-10-02,0,10H80
2010-10-16,1,10H80


In [ ]:
df.info()

# Crime df

In [6]:
crime_df = pd.read_csv(crime).sort_index(ascending=True)

crime_df['Date_Hour'] = pd.to_datetime(crime_df['Date_Hour'])
crime_df['Date_Hour'] = crime_df['Date_Hour'].dt.date

crime_df = crime_df.set_index('Date_Hour')


# selectd columns
crime_df = crime_df[['OffenseType','Hour']]

# # count Offense type given date
# date_crimes = crime_df.groupby(crime_df.index)['OffenseType',].count()

# ## show
# date_crimes.head()

In [7]:
crime_df = pd.get_dummies(crime_df)

In [8]:
crime_df.head()

,Hour,OffenseType_Aggravated Assault,OffenseType_Auto Theft,OffenseType_Burglary,OffenseType_Murder,OffenseType_Rape,OffenseType_Robbery,OffenseType_Theft
Date_Hour,,,,,,,,
2010-01-03,16,0,0,0,0,0,0,1
2010-01-03,17,0,0,1,0,0,0,0
2010-01-03,22,0,0,0,0,0,0,1
2010-01-03,22,0,0,0,0,0,0,1
2010-01-04,17,0,0,0,0,0,0,1


In [9]:
crime_df[crime_df.Hour.isnull()]

,Hour,OffenseType_Aggravated Assault,OffenseType_Auto Theft,OffenseType_Burglary,OffenseType_Murder,OffenseType_Rape,OffenseType_Robbery,OffenseType_Theft
Date_Hour,,,,,,,,


# Days Df
- create a df with everyday timestamp from Jan 2010 to Dec 2017

In [10]:
days = pd.date_range(start='01/01/2010', end='12/31/2017')
days = pd.DataFrame(days)

days.columns = ['days']
days = days.set_index('days').sort_index(ascending=True)
days.head()

""
days
2010-01-01
2010-01-02
2010-01-03
2010-01-04
2010-01-05


In [11]:
365 * 8

2920

In [12]:
len(days)/365

8.005479452054795

In [13]:
days.tail()

""
days
2017-12-27
2017-12-28
2017-12-29
2017-12-30
2017-12-31


# Merge days df with crimes_df

In [14]:
# calendar_crimes = pd.merge(days,date_crimes,  left_index=True, right_index=True, how='left')
# calendar_crimes.head()

In [15]:
calendar_crimes = pd.merge(days,crime_df,  left_index=True, right_index=True, how='left')
calendar_crimes.head()

,Hour,OffenseType_Aggravated Assault,OffenseType_Auto Theft,OffenseType_Burglary,OffenseType_Murder,OffenseType_Rape,OffenseType_Robbery,OffenseType_Theft
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-03,16.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2010-01-03,17.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2010-01-03,22.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
calendar_crimes.info()

In [16]:
len(calendar_crimes[calendar_crimes.Hour.isnull()])

290

In [ ]:
calendar_crimes[calendar_crimes.Hour.isnull()][:10]

## Change nans with 0

In [ ]:
calendar_crimes.OffenseType.replace( np.nan,0, inplace=True)

In [ ]:
calendar_crimes.OffenseType.value_counts(dropna=False)

# merge calendar_crimes with score

In [21]:
merge_data = pd.merge(calendar_crimes,df,  left_index=True, right_index=True, how='left')

# change column names
#merge_data.columns = ['offenses','away_team','win']

merge_data.head()

,Hour,OffenseType_Aggravated Assault,OffenseType_Auto Theft,OffenseType_Burglary,OffenseType_Murder,OffenseType_Rape,OffenseType_Robbery,OffenseType_Theft,win,beat
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-03,16.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN
2010-01-03,17.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN
2010-01-03,22.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN


## Change nan value on beat col`m

In [ ]:
merge_data.beat.value_counts(dropna=False)

In [ ]:
merge_data.beat.replace( np.nan,'10H80', inplace=True)

In [ ]:
merge_data.beat.value_counts(dropna=False)

## Cleanup colums

In [ ]:
merge_data.columns  = ['crime', 'away_team', 'won','beat']

In [ ]:
merge_data.info()

In [ ]:
merge_data.crime.value_counts(dropna=False)

In [ ]:
merge_data.crime = merge_data.crime.astype(int)

In [ ]:
merge_data.crime.value_counts(dropna=False)

In [ ]:
merge_data.head()

# Display non-nans

In [ ]:
merge_data.dropna(subset=['away_team', 'won'])

## add sport col

In [ ]:
merge_data['sport'] = 'cfb'

In [22]:
def game_feature(df):
    if df.win == 1:
        val = 'Won Game'
    elif df.win == 0:
        val = 'Lost Game'
    else:
        val = 'No Game'
    return val

merge_data['game'] = merge_data.apply(game_feature,axis=1)

In [23]:
merge_data.head()

,Hour,OffenseType_Aggravated Assault,OffenseType_Auto Theft,OffenseType_Burglary,OffenseType_Murder,OffenseType_Rape,OffenseType_Robbery,OffenseType_Theft,win,beat,game
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Game
2010-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Game
2010-01-03,16.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,No Game
2010-01-03,17.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,No Game
2010-01-03,22.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,No Game


In [20]:
len(merge_data[merge_data.Hour.isnull()])

290

In [ ]:
merge_data[merge_data.Hour.isnull()][:10]

In [ ]:
merge_data.head()

In [ ]:
#merge_data.to_csv('data/final/Rice1017_final.csv')

In [24]:
merge_data.to_csv('data/final/DUMMYRice1017_final.csv')

In [ ]:
merge_data.info()